In [ ]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/test.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def tokenize_function(example):
    question_string = f"Jaký je sentiment tohoto aspektu: {example['aspect']}?"

    encoding = tokenizer(
        text=example["text"],
        text_pair=question_string,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Map:   0%|          | 0/329 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=3
)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.851790,0.629573
2,0.951100,0.739749,0.699695
3,0.709100,0.723264,0.719512
4,0.709100,0.671847,0.742378
5,0.516700,0.739766,0.760671
6,0.376500,0.801984,0.765244
7,0.273500,0.884977,0.759146
8,0.273500,1.003159,0.780488
9,0.182700,1.118356,0.778963
10,0.132500,1.143480,0.775915


TrainOutput(global_step=1440, training_loss=0.4392566137843662, metrics={'train_runtime': 1593.8759, 'train_samples_per_second': 14.405, 'train_steps_per_second': 0.903, 'total_flos': 3020596204953600.0, 'train_loss': 0.4392566137843662, 'epoch': 10.0})

In [ ]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 1.0031585693359375, 'eval_accuracy': 0.7804878048780488, 'eval_runtime': 12.4395, 'eval_samples_per_second': 52.735, 'eval_steps_per_second': 3.296, 'epoch': 10.0}


In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.0198397636413574, 'eval_accuracy': 0.7416413373860182, 'eval_runtime': 6.3575, 'eval_samples_per_second': 51.75, 'eval_steps_per_second': 3.303, 'epoch': 10.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA")

('/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA/spm.model',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA/added_tokens.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA/tokenizer.json')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/deberta-v3-baseQA")

## Vyhodnocovací metriky

In [ ]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [ ]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.0198396444320679, 'eval_model_preparation_time': 0.0033, 'eval_accuracy': 0.7416413373860182, 'eval_runtime': 7.1644, 'eval_samples_per_second': 45.922, 'eval_steps_per_second': 5.862}


In [ ]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [ ]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.0198396444320679, 'eval_model_preparation_time': 0.0047, 'eval_f1': 0.7384139199851674, 'eval_runtime': 7.4463, 'eval_samples_per_second': 44.183, 'eval_steps_per_second': 5.64}


In [ ]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [ ]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.0198396444320679, 'eval_model_preparation_time': 0.0055, 'eval_f1': 0.7311215150347458, 'eval_runtime': 6.6345, 'eval_samples_per_second': 49.589, 'eval_steps_per_second': 6.331}


## Praktické testy

In [ ]:
def predict_sentiment(model, tokenizer, aspect, text):
    question_string = f"Jaký je sentiment tohoto aspektu: {aspect}?"

    encoding = tokenizer(
        text=text,
        text_pair=question_string,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )

    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [ ]:
example_aspect = "výkon"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "displej"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "baterie"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-1.9261, -1.4841,  4.6409]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 3.3784, -2.6051, -1.2310]])
Predicted sentiment for L2: negative, expected: negative
tensor([[-1.9336, -1.3866,  4.6218]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [ ]:
example_aspect = "personál"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "stůl"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "jídlo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "pivo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 2.0061, -2.3991,  0.2539]])
Predicted sentiment for R1: negative, expected: negative
tensor([[-0.2840, -0.6230,  1.3024]])
Predicted sentiment for R2: positive, expected: negative
tensor([[-2.1916, -1.0930,  4.6430]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-2.1614, -1.0819,  4.6117]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [ ]:
example_aspect = "Ósace"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[ 0.6973, -0.5245,  0.0033]])
Predicted sentiment for M1: negative, expected: neutral
tensor([[-1.0647, -0.2769,  2.2982]])
Predicted sentiment for M2: positive, expected: positive
tensor([[ 2.2508, -1.2396, -1.2066]])
Predicted sentiment for M3: negative, expected: negative


### Sada O

In [ ]:
example_aspect = "den"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "počasí"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-2.1461, -1.3218,  4.7330]])
Predicted sentiment for O1: positive, expected: positive
tensor([[ 3.2137, -1.6991, -1.9207]])
Predicted sentiment for O2: negative, expected: negative
